# Very very Very important information.

* I dropped large dataset for smoothness of my programm.
* belongs_to_collection:- this column contain more than 40k null values. So if your machine is powerfull then you dropped or ignore this column in initial stage.
* Because when i droped it and droped null value my pickle size was 15GB+ and when i tried to run streamlit locally it occurs memory error.
* I this case i use `belongs_to_collection` and changed name to `keywords`.
  ## Why did i contain `keywords` Columns.
  * Because my streamlit strugging not running, computer hanged crashed so i include `keywords` columns and dropped all na values.
  * then size decreased (45K to  4.4K+) and now running well.
* In real project where we have machine that can handle this data eaisly in that we will ignore than columns.

# This is Content based recommendation system.
### Three Types of Recommendatino system
1. Content based Recommendation system. :- Based on user. (example you previously likes or comments.)
2. Collaborative Filtering. :- Based on same taste of people. Video you watch and same video other user watch, and after that most of the user watched that video, so that video also suggested to you.
3. Hybrid Systems. :- Mix of Both.


## WorkFlow overview
1. Data collection and preprocessing
2. Model
3. Website
4. Deploy

In [1]:
import pandas as pd
import numpy as np
import ast
import sklearn 
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

In [2]:
# Data Downloaded from Kaggle.
movies = pd.read_csv("C:\\Users\\BKJST\\Desktop\\python\\Machine learning\\Project\\Advance project\\Movies Recommendation system complete ombd\\movies_metadata.csv")
credits = pd.read_csv('C:\\Users\\BKJST\\Desktop\\python\\Machine learning\\Project\\Advance project\\Movies Recommendation system complete ombd\\credits.csv')
movies.head(1)

C:\Users\BKJST\AppData\Local\Temp\ipykernel_26188\3000923487.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("C:\\Users\\BKJST\\Desktop\\python\\Machine learning\\Project\\Advance project\\Movies Recommendation system complete ombd\\movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [3]:
# check columns
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
# columns information (like null value data type etc)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
movies[movies['belongs_to_collection'].notnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
12,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",NaN,21032,tt0112453,en,Balto,An outcast half-wolf risks his life to prevent...,...,1995-12-22,11348324.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Part Dog. Part Wolf. All Hero.,Balto,False,7.1,423.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45355,False,"{'id': 37261, 'name': 'The Carry On Collection...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,24568,tt0061680,en,Follow That Camel,Bertram Oliphant West (also known as Bo West) ...,...,1967-09-01,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Carry On Follow That Camel,False,6.1,19.0
45358,False,"{'id': 37261, 'name': 'The Carry On Collection...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,19307,tt0064133,en,Carry On Camping,Sid and Bernie keep having their amorous inten...,...,1969-05-29,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fun and games in the great outdoors!,Carry On Camping,False,6.5,25.0
45369,False,"{'id': 37261, 'name': 'The Carry On Collection...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,21251,tt0074286,en,Carry On England,Captain S. Melly takes over as the new Command...,...,1976-10-01,0.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Carry On England,False,4.8,15.0
45371,False,"{'id': 477208, 'name': 'DC Super Hero Girls Co...",0,"[{'id': 16, 'name': 'Animation'}]",NaN,460135,tt7158814,en,LEGO DC Super Hero Girls: Brain Drain,"When Supergirl, Wonder Woman, Batgirl, Bumbleb...",...,2017-08-30,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,LEGO DC Super Hero Girls: Brain Drain,False,10.0,2.0


In [6]:
movies[movies['belongs_to_collection'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [7]:
# Renaming columns belongs_to_collection to keywords
movies.rename(columns= {'belongs_to_collection':'keywords'}, inplace = True)

In [8]:
movies.head(1)

,adult,keywords,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [9]:
# Create new dataframe with required columns.
movies_1 = movies[['id', 'imdb_id', 'overview','title', 'genres','keywords']]
movies_1.isnull().sum()

id              0
imdb_id        17
overview      954
title           6
genres          0
keywords    40972
dtype: int64

In [10]:
movies_2 = movies_1.dropna() # Now let's drop null values and store in new dataframe.
movies_2.isnull().sum() 

id          0
imdb_id     0
overview    0
title       0
genres      0
keywords    0
dtype: int64

In [11]:
for i in movies_2['id']:
    print(i)

862
15602
11862
710
21032
9273
8012
9598
9312
10530
11525
10634
755
9102
10874
27793
9737
414
5894
8839
8068
1572
13552
9073
9070
1642
11980
10802
10149
9348
3512
50797
76
2292
8467
15789
628
11
63105
527
18220
110
108
109
22586
11395
9059
193
3049
9331
9495
888
8587
11596
854
36593
11011
1637
8831
11667
75555
8011
2758
9409
306
11310
24257
1634
5503
2019
9255
16097
329
11231
5550
11236
8850
78
9066
771
812
280
268
274
10020
10824
25059
11827
8766
19042
954
8840
25087
5801
896
8072
20762
11687
9323
532
10547
77771
627
602
10545
9327
12606
10546
9431
10061
238
11134
38955
19004
10987
62
3529
10331
39448
10680
18660
35115
14821
14136
10869
19186
6878
15944
11224
15947
10948
11229
562
10163
11238
88
8491
402
25750
11843
1103
11298
4480
4481
531
36555
11455
1891
85
679
429
1892
348
766
539
525
240
530
9277
218
829
765
105
8009
756
11314
89
8077
41671
27214
52263
33519
16235
11449
41848
8851
229
9529
7340
377
794
145
199
9279
17204
11674
1573
152
174
172
154
157
168
364
11967
9086
621
9037


In [12]:
credits.id

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45471    439050
45472    111109
45473     67758
45474    227506
45475    461257
Name: id, Length: 45476, dtype: int64

In [13]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [14]:
credits['id'][0]

np.int64(862)

In [15]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [16]:
# Before Merge lets check shape
movies.shape, credits.shape

((45466, 24), (45476, 3))

In [17]:
movies_2['id'] = movies_2.id.astype(int)
movies_2.id

C:\Users\BKJST\AppData\Local\Temp\ipykernel_26188\247471114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_2['id'] = movies_2.id.astype(int)


0           862
2         15602
4         11862
9           710
12        21032
          ...  
45355     24568
45358     19307
45369     21251
45371    460135
45382    182030
Name: id, Length: 4429, dtype: int64

In [18]:
# lets merge both df and create new df.
movies_3 = movies_2.merge(credits, on = 'id')
movies_3

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","{'id': 10194, 'name': 'Toy Story Collection', ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,15602,tt0113228,A family wedding reignites the ancient feud be...,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
2,11862,tt0113041,Just when George Banks has recovered from his ...,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","{'id': 96871, 'name': 'Father of the Bride Col...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
3,710,tt0113189,James Bond must unmask the mysterious head of ...,GoldenEye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","{'id': 645, 'name': 'James Bond Collection', '...","[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."
4,21032,tt0112453,An outcast half-wolf risks his life to prevent...,Balto,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...","{'id': 117693, 'name': 'Balto Collection', 'po...","[{'cast_id': 1, 'character': 'Balto (voice)', ...","[{'credit_id': '593f24b9c3a3680369002371', 'de..."
...,...,...,...,...,...,...,...,...
4433,24568,tt0061680,Bertram Oliphant West (also known as Bo West) ...,Carry On Follow That Camel,"[{'id': 35, 'name': 'Comedy'}]","{'id': 37261, 'name': 'The Carry On Collection...","[{'cast_id': 3, 'character': 'Sergeant Nocker'...","[{'credit_id': '52fe449dc3a368484e02d0f7', 'de..."
4434,19307,tt0064133,Sid and Bernie keep having their amorous inten...,Carry On Camping,"[{'id': 35, 'name': 'Comedy'}]","{'id': 37261, 'name': 'The Carry On Collection...","[{'cast_id': 3, 'character': 'Sid Boggle', 'cr...","[{'credit_id': '52fe47cf9251416c750a61d7', 'de..."
4435,21251,tt0074286,Captain S. Melly takes over as the new Command...,Carry On England,"[{'id': 35, 'name': 'Comedy'}]","{'id': 37261, 'name': 'The Carry On Collection...","[{'cast_id': 4, 'character': 'Captain S. Melly...","[{'credit_id': '52fe4411c3a368484e00da03', 'de..."
4436,460135,tt7158814,"When Supergirl, Wonder Woman, Batgirl, Bumbleb...",LEGO DC Super Hero Girls: Brain Drain,"[{'id': 16, 'name': 'Animation'}]","{'id': 477208, 'name': 'DC Super Hero Girls Co...","[{'cast_id': 0, 'character': 'Wonder Woman', '...","[{'credit_id': '593136c092514115e500aafd', 'de..."


In [19]:
movies_3.shape

(4438, 8)

In [20]:
movies_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4438 entries, 0 to 4437
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4438 non-null   int64 
 1   imdb_id   4438 non-null   object
 2   overview  4438 non-null   object
 3   title     4438 non-null   object
 4   genres    4438 non-null   object
 5   keywords  4438 non-null   object
 6   cast      4438 non-null   object
 7   crew      4438 non-null   object
dtypes: int64(1), object(7)
memory usage: 277.5+ KB


In [21]:
movies_3.isnull().sum()

id          0
imdb_id     0
overview    0
title       0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [22]:
# Currently my data is in not clear, it is in dictionary format, stirng so we will extract our required data.
movies_3['crew'][0] # we will extract data from this and will create a tag
# i will extract producer name from this.

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [23]:
movies_3['cast'][0] # we will extract actor name form this we will crate function 

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [24]:
movies_3.overview[2]

"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."

In [25]:
movies_3.keywords[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [26]:
movies_3.iloc[2].genres

"[{'id': 35, 'name': 'Comedy'}]"

In [27]:
# so we saw data, have in different different format
# we have clean it.

import ast # for string it will convert string to list.
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]')



[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 80, 'name': 'Crime'}]

In [28]:
# lets create a function for 

def convert(obj):
    L = [] # empty list
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [29]:
movies_3['genres'] = movies_3.genres.apply(convert)
movies_3.head(3) # we converted it into a list by the help of this function.

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"[Animation, Comedy, Family]","{'id': 10194, 'name': 'Toy Story Collection', ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,15602,tt0113228,A family wedding reignites the ancient feud be...,Grumpier Old Men,"[Romance, Comedy]","{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
2,11862,tt0113041,Just when George Banks has recovered from his ...,Father of the Bride Part II,[Comedy],"{'id': 96871, 'name': 'Father of the Bride Col...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [30]:
movies_3.keywords.dtypes

dtype('O')

In [31]:
names = []

for item in movies_3.keywords:
    try:
        parsed = ast.literal_eval(item)
        name = parsed.get('name')
        if name:
            names.append(name)
    except Exception as e:
        # Handle cases where parsing fails
        print(f"Skipping entry due to error: {e}")

# Now, names contains all extracted 'name' values
print(f"Extracted {len(names)} names")

Extracted 4438 names


In [32]:
movies_3.keywords = names

In [33]:
# movies_3['keywords'] = movies_3.keywords.apply(convert)
movies_3.head() # also converted it into a list.

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"[Animation, Comedy, Family]",Toy Story Collection,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,15602,tt0113228,A family wedding reignites the ancient feud be...,Grumpier Old Men,"[Romance, Comedy]",Grumpy Old Men Collection,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
2,11862,tt0113041,Just when George Banks has recovered from his ...,Father of the Bride Part II,[Comedy],Father of the Bride Collection,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
3,710,tt0113189,James Bond must unmask the mysterious head of ...,GoldenEye,"[Adventure, Action, Thriller]",James Bond Collection,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."
4,21032,tt0112453,An outcast half-wolf risks his life to prevent...,Balto,"[Family, Animation, Adventure]",Balto Collection,"[{'cast_id': 1, 'character': 'Balto (voice)', ...","[{'credit_id': '593f24b9c3a3680369002371', 'de..."


In [34]:
# defining a function for extract name from cast.
def convert_cast(cast):
    L = []
    counter = 0
    for i in ast.literal_eval(cast):
        if counter !=3: # it will check till 3 because in 3rd number have main actor name.
            L.append(i['name']) # extract name
            counter +=1
        else:
            break
    return L

In [35]:
movies_3['cast'] = movies_3['cast'].apply(convert_cast)
movies_3.head()

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"[Animation, Comedy, Family]",Toy Story Collection,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,15602,tt0113228,A family wedding reignites the ancient feud be...,Grumpier Old Men,"[Romance, Comedy]",Grumpy Old Men Collection,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
2,11862,tt0113041,Just when George Banks has recovered from his ...,Father of the Bride Part II,[Comedy],Father of the Bride Collection,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
3,710,tt0113189,James Bond must unmask the mysterious head of ...,GoldenEye,"[Adventure, Action, Thriller]",James Bond Collection,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."
4,21032,tt0112453,An outcast half-wolf risks his life to prevent...,Balto,"[Family, Animation, Adventure]",Balto Collection,"[Kevin Bacon, Bob Hoskins, Bridget Fonda]","[{'credit_id': '593f24b9c3a3680369002371', 'de..."


In [36]:
# now we resolve all column left crew. lets fix it.
# define a function to get crew name.
def get_director(crew):
    L = []
    for i in ast.literal_eval(crew):
        L.append(i['name'])
        break

    return L

In [37]:
movies_3['crew'] = movies_3['crew'].apply(get_director)
movies_3.head() # we fix all column issue.

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"[Animation, Comedy, Family]",Toy Story Collection,"[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter]
1,15602,tt0113228,A family wedding reignites the ancient feud be...,Grumpier Old Men,"[Romance, Comedy]",Grumpy Old Men Collection,"[Walter Matthau, Jack Lemmon, Ann-Margret]",[Howard Deutch]
2,11862,tt0113041,Just when George Banks has recovered from his ...,Father of the Bride Part II,[Comedy],Father of the Bride Collection,"[Steve Martin, Diane Keaton, Martin Short]",[Alan Silvestri]
3,710,tt0113189,James Bond must unmask the mysterious head of ...,GoldenEye,"[Adventure, Action, Thriller]",James Bond Collection,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]",[Martin Campbell]
4,21032,tt0112453,An outcast half-wolf risks his life to prevent...,Balto,"[Family, Animation, Adventure]",Balto Collection,"[Kevin Bacon, Bob Hoskins, Bridget Fonda]",[Steven Spielberg]


In [38]:
# convert overview column from stirng to list.
movies_3['overview'] = movies_3.overview.apply(lambda x: x.split())
movies_3.head()

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"[Led, by, Woody,, Andy's, toys, live, happily,...",Toy Story,"[Animation, Comedy, Family]",Toy Story Collection,"[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter]
1,15602,tt0113228,"[A, family, wedding, reignites, the, ancient, ...",Grumpier Old Men,"[Romance, Comedy]",Grumpy Old Men Collection,"[Walter Matthau, Jack Lemmon, Ann-Margret]",[Howard Deutch]
2,11862,tt0113041,"[Just, when, George, Banks, has, recovered, fr...",Father of the Bride Part II,[Comedy],Father of the Bride Collection,"[Steve Martin, Diane Keaton, Martin Short]",[Alan Silvestri]
3,710,tt0113189,"[James, Bond, must, unmask, the, mysterious, h...",GoldenEye,"[Adventure, Action, Thriller]",James Bond Collection,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]",[Martin Campbell]
4,21032,tt0112453,"[An, outcast, half-wolf, risks, his, life, to,...",Balto,"[Family, Animation, Adventure]",Balto Collection,"[Kevin Bacon, Bob Hoskins, Bridget Fonda]",[Steven Spielberg]


In [39]:
# now we have to remove space otherwise it will crate problem.
# see like kinds of problem : My name is Bablu Jha. other may be Bablu  Mishara etc.
# if someone search bablu then result will different result will not proper

In [40]:
movies_3['genres'] = movies_3['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies_3['keywords'] = movies_3['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies_3['cast'] = movies_3['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies_3['crew'] = movies_3['crew'].apply(lambda x: [i.replace(" ","") for i in x])

movies_3.head() # all space removed-> this is now in desired format.

,id,imdb_id,overview,title,genres,keywords,cast,crew
0,862,tt0114709,"[Led, by, Woody,, Andy's, toys, live, happily,...",Toy Story,"[Animation, Comedy, Family]","[T, o, y, , S, t, o, r, y, , C, o, l, l, e, c,...","[TomHanks, TimAllen, DonRickles]",[JohnLasseter]
1,15602,tt0113228,"[A, family, wedding, reignites, the, ancient, ...",Grumpier Old Men,"[Romance, Comedy]","[G, r, u, m, p, y, , O, l, d, , M, e, n, , C, ...","[WalterMatthau, JackLemmon, Ann-Margret]",[HowardDeutch]
2,11862,tt0113041,"[Just, when, George, Banks, has, recovered, fr...",Father of the Bride Part II,[Comedy],"[F, a, t, h, e, r, , o, f, , t, h, e, , B, r, ...","[SteveMartin, DianeKeaton, MartinShort]",[AlanSilvestri]
3,710,tt0113189,"[James, Bond, must, unmask, the, mysterious, h...",GoldenEye,"[Adventure, Action, Thriller]","[J, a, m, e, s, , B, o, n, d, , C, o, l, l, e,...","[PierceBrosnan, SeanBean, IzabellaScorupco]",[MartinCampbell]
4,21032,tt0112453,"[An, outcast, half-wolf, risks, his, life, to,...",Balto,"[Family, Animation, Adventure]","[B, a, l, t, o, , C, o, l, l, e, c, t, i, o, n]","[KevinBacon, BobHoskins, BridgetFonda]",[StevenSpielberg]


In [41]:
# now we will create a tag.
movies_4 = movies_3[['imdb_id', 'title']]
movies_4.head() # i will add tag columns in this.

,imdb_id,title
0,tt0114709,Toy Story
1,tt0113228,Grumpier Old Men
2,tt0113041,Father of the Bride Part II
3,tt0113189,GoldenEye
4,tt0112453,Balto


In [42]:
movies_4['tags'] = movies_3['overview'] + movies_3['genres'] + movies_3['cast'] + movies_3['crew']
movies_4.head()

C:\Users\BKJST\AppData\Local\Temp\ipykernel_26188\2684153915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_4['tags'] = movies_3['overview'] + movies_3['genres'] + movies_3['cast'] + movies_3['crew']


,imdb_id,title,tags
0,tt0114709,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,..."
1,tt0113228,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ..."
2,tt0113041,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr..."
3,tt0113189,GoldenEye,"[James, Bond, must, unmask, the, mysterious, h..."
4,tt0112453,Balto,"[An, outcast, half-wolf, risks, his, life, to,..."


In [43]:
# now convert tags into string.
movies_4['tags'] = movies_4['tags'].apply(lambda x: " ".join(x))
movies_4.head()

C:\Users\BKJST\AppData\Local\Temp\ipykernel_26188\1450811924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_4['tags'] = movies_4['tags'].apply(lambda x: " ".join(x))


,imdb_id,title,tags
0,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...
2,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...
3,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...
4,tt0112453,Balto,An outcast half-wolf risks his life to prevent...


In [44]:
movies_4['tags'][0] # now it is in string format.

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. Animation Comedy Family TomHanks TimAllen DonRickles JohnLasseter"

In [45]:
# now convert all tags in lower case.
# lower case is recomended so we will use it.
movies_4['tags'] = movies_4['tags'].apply(lambda x: x.lower())
movies_4.head()

C:\Users\BKJST\AppData\Local\Temp\ipykernel_26188\1862736801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_4['tags'] = movies_4['tags'].apply(lambda x: x.lower())


,imdb_id,title,tags
0,tt0114709,Toy Story,"led by woody, andy's toys live happily in his ..."
1,tt0113228,Grumpier Old Men,a family wedding reignites the ancient feud be...
2,tt0113041,Father of the Bride Part II,just when george banks has recovered from his ...
3,tt0113189,GoldenEye,james bond must unmask the mysterious head of ...
4,tt0112453,Balto,an outcast half-wolf risks his life to prevent...


In [46]:
final_movies = movies_4.copy()
final_movies.head()

,imdb_id,title,tags
0,tt0114709,Toy Story,"led by woody, andy's toys live happily in his ..."
1,tt0113228,Grumpier Old Men,a family wedding reignites the ancient feud be...
2,tt0113041,Father of the Bride Part II,just when george banks has recovered from his ...
3,tt0113189,GoldenEye,james bond must unmask the mysterious head of ...
4,tt0112453,Balto,an outcast half-wolf risks his life to prevent...


In [47]:
# now we have to find out similarity between text.
final_movies.tags[0]

"led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences. animation comedy family tomhanks timallen donrickles johnlasseter"

In [48]:
final_movies.tags[1]

"a family wedding reignites the ancient feud between next-door neighbors and fishing buddies john and max. meanwhile, a sultry italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. but she's less interested in seafood than she is in cooking up a hot time with max. romance comedy waltermatthau jacklemmon ann-margret howarddeutch"

In [49]:
# if we find manually it will takes lots of time and chances of eror.
# so we will convert it in vector by the help of scikit learn.

# Vectorization
import sklearn
from sklearn.feature_extraction.text import CountVectorizer


cv = CountVectorizer(
    max_features = 5000, stop_words = 'english'
) # stop_words means that the words used for sentence fromation like  is are, am, as etc.
cv

CountVectorizer(max_features=5000, stop_words='english')

In [50]:
dir(cv)

['_CountVectorizer__metadata_request__fit',
 '_CountVectorizer__metadata_request__transform',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_request_for_signature',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_feature_names',
 '_check_n_features',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_parameter_constraints',
 '_r

In [51]:
vector = cv.fit_transform(final_movies['tags']).toarray()
vector # it convert into numerical form and we store it in numpy array.

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [52]:
vector.shape 

(4438, 5000)

In [53]:
vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
vector_feature_names = cv.get_feature_names_out()
vector_feature_names

array(['000', '007', '10', ..., 'zone', 'är', 'он'], dtype=object)

In [55]:
# we are unable to see all the value.
# lets see by using loop
for i in vector_feature_names:
    print(i) # here some value is duplicate value like actors and actor like this many.

000
007
10
100
11
12
13
13th
14
15
16
17
17th
18
1930
1940
1945
1950
1954
1960
1967
1969
1970
1980
1980s
1984
1985
1986
1990
1992
1997
1998
1999
19th
20
200
2000
2001
2002
2005
2006
2007
2008
2009
2010
2011
20th
21st
22nd
23
24
25
30
300
3d
3rd
40
4th
50
60
aaron
abandoned
abby
abducted
abducts
abhishekbachchan
abilities
ability
able
aboard
abroad
absent
abuse
academy
accept
accepted
accepts
access
accident
accidentally
accidents
accompanied
accomplish
according
account
accused
ace
achieve
acquire
act
acting
action
actions
activities
activity
actor
actors
actress
acts
actual
actually
adam
adapt
adaptation
add
addition
adds
aditya
adopted
adorable
adult
adulthood
adults
advanced
advantage
adventure
adventurer
adventures
advice
affair
affairs
afghanistan
afraid
africa
african
afro
aftermath
age
aged
agency
agenda
agent
agents
ages
aggressive
aging
ago
agree
agrees
ahead
aid
aided
aims
air
airplane
airport
aka
akane
akemi
akikaurismäki
akiratoriyama
akshaykumar
al
alaindelon
alan
alansilv

In [56]:
# now lets fix that duplicate data otherwise it will affect result.
# for this i will use nltk library.
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps

<PorterStemmer>

In [57]:
# simply it will convert in Verb_1 form.
# like play, played, playing, plays for all words it will use play.

ps.stem('play'), ps.stem('played'), ps.stem('playing'), ps.stem('plays')

('play', 'play', 'play', 'play')

In [58]:
# define funciton that will remvoe duplicate value.

def stem(text):
    L = []

    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L) # converting in string.

In [59]:
final_movies['tags'] = final_movies['tags'].apply(stem)
# after this rerun above process form sklearn or rewrite code.

In [60]:
vector = cv.fit_transform(final_movies['tags']).toarray()
vector # it convert into numerical form and we store it in numpy array.

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [61]:
vector_feature_names = cv.get_feature_names_out()
vector_feature_names # it assing into vectors

array(['000', '007', '10', ..., 'zone', 'är', 'он'], dtype=object)

In [62]:
for i in vector_feature_names:
    # print(type(i))
    print(i) # now almost all duplicte value rmoved.

000
007
10
100
11
12
13
13th
14
15
16
17
17th
18
1930
1940
1945
1950
1954
1960
1967
1969
1970
1980
1984
1985
1986
1990
1992
1997
1998
1999
19th
20
200
2000
2001
2002
2005
2006
2007
2008
2009
2010
2011
20th
21st
22nd
23
24
25
30
300
3d
3rd
40
4th
50
60
70
90
aaron
abandon
abduct
abhishekbachchan
abil
abl
aboard
abov
abroad
absent
absolut
absorb
abus
academi
academy
accept
access
accid
accident
accompani
accomplish
accord
account
accus
ace
achiev
acquaint
acquir
act
action
activ
activities
actor
actress
actual
ad
adam
adapt
add
addict
addit
aditya
adjust
admir
adopt
ador
adult
adulthood
advanc
advantag
adventur
adventure
adventures
advic
advis
affair
affect
affection
afflict
afghanistan
afraid
africa
african
afro
aftermath
ag
age
agenc
agency
agenda
agent
agents
aggress
aging
ago
agre
ahead
aid
aim
air
airplan
airport
ajaydevgn
aka
akan
akemi
akikaurismäki
akiratoriyama
akshaykumar
al
aladdin
alaindelon
alainpoiré
alan
alansilvestri
alarm
alaska
albertpyun
albin
alcohol
alec
aleksandrdem

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)
similarity.shape

(4438, 4438)

In [64]:
similarity[10]

array([0.04950738, 0.        , 0.11595421, ..., 0.10090092, 0.08346223,
       0.08084521])

In [65]:
sorted(similarity[0])

[np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float6

In [66]:
sorted(similarity[0],reverse = True) # showing difference between two movies. it will recommed more near movies according to simliarity.
# when we sorted it, problems occurs. index number is changed.

[np.float64(1.0000000000000004),
 np.float64(0.42505223079887944),
 np.float64(0.40201512610368495),
 np.float64(0.2842676218074806),
 np.float64(0.26516504294495535),
 np.float64(0.26516504294495535),
 np.float64(0.25476233268253407),
 np.float64(0.25116550748367644),
 np.float64(0.24549512651549144),
 np.float64(0.2452889452598031),
 np.float64(0.24212399923954658),
 np.float64(0.24122532033850386),
 np.float64(0.2405626121623441),
 np.float64(0.22963966338592295),
 np.float64(0.22821773229381925),
 np.float64(0.22821773229381925),
 np.float64(0.2269637792195609),
 np.float64(0.2254938084008487),
 np.float64(0.22271770159368703),
 np.float64(0.222717701593687),
 np.float64(0.22176638128637188),
 np.float64(0.22017621148199487),
 np.float64(0.21997067253202995),
 np.float64(0.21957751641342002),
 np.float64(0.21808918607248967),
 np.float64(0.2175970699446223),
 np.float64(0.2175970699446223),
 np.float64(0.2175970699446223),
 np.float64(0.2165063509461097),
 np.float64(0.216506350946

In [67]:
# so in this case we will use enumerate
# we can not apply directly enumerate(similarity[0])
# before it use list
list(enumerate(similarity[0]))

# nor apply sorting
sorted(list(enumerate(similarity[0])),reverse = True, key = lambda x: x[1])
# now it is in proper format.

[(0, np.float64(1.0000000000000004)),
 (518, np.float64(0.42505223079887944)),
 (1997, np.float64(0.40201512610368495)),
 (801, np.float64(0.2842676218074806)),
 (1132, np.float64(0.26516504294495535)),
 (2657, np.float64(0.26516504294495535)),
 (3199, np.float64(0.25476233268253407)),
 (323, np.float64(0.25116550748367644)),
 (4250, np.float64(0.24549512651549144)),
 (4045, np.float64(0.2452889452598031)),
 (3440, np.float64(0.24212399923954658)),
 (3534, np.float64(0.24122532033850386)),
 (4226, np.float64(0.2405626121623441)),
 (1726, np.float64(0.22963966338592295)),
 (564, np.float64(0.22821773229381925)),
 (4218, np.float64(0.22821773229381925)),
 (1854, np.float64(0.2269637792195609)),
 (4299, np.float64(0.2254938084008487)),
 (2350, np.float64(0.22271770159368703)),
 (1609, np.float64(0.222717701593687)),
 (1640, np.float64(0.22176638128637188)),
 (2561, np.float64(0.22017621148199487)),
 (3253, np.float64(0.21997067253202995)),
 (4205, np.float64(0.21957751641342002)),
 (1322,

In [68]:
# first 5 movies based on similarity.

sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x: x[1])[1:6]

[(518, np.float64(0.42505223079887944)),
 (1997, np.float64(0.40201512610368495)),
 (801, np.float64(0.2842676218074806)),
 (1132, np.float64(0.26516504294495535)),
 (2657, np.float64(0.26516504294495535))]

In [69]:
def recommendation(movie):
    if movie not in final_movies['title'].values: # i am sure all movie will be in database which will be provided in list.
        print(f'Movie {movie} not found our database, please check movie name.')

        
    movie_index = final_movies[final_movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x: x[1])[1:6]

    for i in movies_list:
        print(final_movies.iloc[i[0]].title)

    

In [70]:
final_movies.head()

,imdb_id,title,tags
0,tt0114709,Toy Story,"led by woody, andy' toy live happili in hi roo..."
1,tt0113228,Grumpier Old Men,a famili wed reignit the ancient feud between ...
2,tt0113041,Father of the Bride Part II,just when georg bank ha recov from hi daughter...
3,tt0113189,GoldenEye,jame bond must unmask the mysteri head of the ...
4,tt0112453,Balto,an outcast half-wolf risk hi life to prevent a...


In [71]:
recommendation('Avatar')

Avatar 2
Xtro 3: Watch the Skies
Gamera vs. Guiron
Mad Max Beyond Thunderdome
Godzilla, Mothra and King Ghidorah: Giant Monsters All-Out Attack


In [72]:
final_movies.iloc[1213]

imdb_id                                            tt0045177
title                                        Son of Paleface
tags       in thi sequel to "the paleface", peter potter ...
Name: 1213, dtype: object

In [73]:
final_movies.iloc[1213].title

'Son of Paleface'

In [74]:
recommendation('Toy Story')

Toy Story 2
Toy Story 3
Harry Potter and the Philosopher's Stone
The Inheritance
Niko 2 - Little Brother, Big Trouble


In [75]:
import pickle # for dump pickle for production.

In [76]:
final_movies['title'].values # we will use it to create a list.

array(['Toy Story', 'Grumpier Old Men', 'Father of the Bride Part II',
       ..., 'Carry On England', 'LEGO DC Super Hero Girls: Brain Drain',
       'The Twin Swords'], dtype=object)

In [77]:
pickle.dump(final_movies.to_dict(), open('movie_dict.pkl', 'wb'))

In [78]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))